In [1]:
import tensorflow as tf
import numpy as np
import gym
from pylab import *
import scipy.misc

In [2]:
%matplotlib notebook

In [134]:
N_ACTIONS = 9
actions = np.arange(N_ACTIONS)

In [135]:
tf.reset_default_graph()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

s_op = tf.placeholder(tf.float32, [None, 84, 84, 4])
r_op = tf.placeholder(tf.float32, [None])

p = {}
pi = {}

for scope in ['network1', 'network2']:
    with tf.variable_scope(scope):
        x = tf.layers.conv2d(
                inputs=s_op,
                filters=32,
                kernel_size=8,
                strides=4,
                activation=tf.nn.relu)

        x = tf.layers.conv2d(
                inputs=x,
                filters=64,
                kernel_size=4,
                strides=2,
                activation=tf.nn.relu)

        x = tf.layers.conv2d(
                inputs=x,
                filters=64,
                kernel_size=3,
                strides=1,
                activation=tf.nn.relu)

        w, h, f = x.shape[1:]
        x = tf.reshape(x, [-1, int(w * h * f)])

        x = tf.layers.dense(
                inputs=x,
                units=512,
                activation=tf.nn.relu)

        a_logits = tf.layers.dense(
                inputs=x,
                units=N_ACTIONS,
                activation=None)

        if scope == 'network1':
            a_softmax = tf.nn.softmax(a_logits)
            a_max = tf.reduce_max(a_softmax, axis=1)
        else:
            v = tf.layers.dense(
                inputs=x,
                units=1,
                activation=None)

#lp = tf.log(a_softmax[at]) * (r_op - v)
                              
#t = tf.train.AdamOptimizer().minimize(loss)

NameError: name 'at' is not defined

In [ ]:
l = tf.losses.log_losses()

In [31]:
def copy_network():
    n1_dict = {var.name: var for var in tf.trainable_variables() if 'network1' in var.name}
    n2_dict = {var.name: var for var in tf.trainable_variables() if 'network2' in var.name}
    copy_ops = []
    for name, var in n2_dict.items():
        op = var.assign(n1_dict[name.replace('network2', 'network1')].value())
        copy_ops.append(op)
    sess.run(copy_ops)

In [126]:
sess.run(tf.global_variables_initializer())

In [92]:
def with_prob(p):
    if np.random.random() < p:
        return True
    else:
        return False

epsilon = 0.1
g = 0.9

In [104]:
class DummyEnv:
    
    action_space = gym.spaces.discrete.Discrete(2)
    
    def reset(self):
        self.i = 0
        self.reward = 0
        return np.zeros(shape=(84, 84, 4))
    def step(self, a):
        if self.i == 5:
            done = True
            reward = self.reward
        else:
            done = False
            reward = 0
            self.reward += a
            self.i += 1
        return np.zeros(shape=(84, 84, 4)), reward, done, None

In [18]:
env = gym.make('Enduro-v0')

[2017-07-13 19:28:53,748] Making new env: Enduro-v0


In [105]:
env = DummyEnv()
env.reset()
done = False
while not done:
    a = env.action_space.sample()
    print("Took action", a)
    _, reward, done, _ = env.step(a)
    print("Got reward ", reward)

Took action 1
Got reward  0
Took action 1
Got reward  0
Took action 0
Got reward  0
Took action 0
Got reward  0
Took action 1
Got reward  0
Took action 0
Got reward  3


In [131]:
t = 0
t_max = 100

s = np.zeros(10)
r1 = np.zeros(10)
a = np.zeros(10)
r2_list = []

for i in range(10):
    t_start = t
    s[t] = env.reset()
    done = False
    
    while True:
        a_p = sess.run(ap_op, feed_dict={s_op: s})
        a[t] = np.random.choice(ACTIONS, p=a_p)
        s[t+1], r[t], done, _ = env.step(a[t])
        t += 1        
    if done or t - t_start == t_max:
        break
    
    if done:
        r2 = 0
    else:
        r2 = sess.run(v_op, feed_dict={s_op: s[t]})
    
    for i in range(t-1, t_start-1, -1):
        r2_list.append(r[i] + g * r2)
    
    
    sess.run(t1, feed_dict={s_op: s, big_r_op: r})
    sess.run(t2, feed_dict={s_op: s, big_r_op: r})

0
0
0
0
0
0
0
0
0
0


In [110]:
observation.shape
action = env.action_space.sample()
os = []
for i in range(4):
    o1, _, _, _ = env.step(action)
    o2, _, _, _ = env.step(action)
    o = np.maximum(o1, o2)
    o = np.mean(o, axis=2)
    o = scipy.misc.imresize(o, (84, 84))
    os.append(o)
os = np.stack(os, axis=-1)

In [80]:
os = np.resize(os, (1, 84, 84, 4))
print(os.shape)
print(sess.run(q['network1'], feed_dict={s: os}))
print(sess.run(qi['network1'], feed_dict={s: os}))

(1, 84, 84, 4)
[[  0.69894785  -0.22462213   5.59576368  -7.67884588  16.15483665
    2.82746601   0.95167667   5.31193829   0.30126441]]
[ 16.15483665]
[4]
